# Job Recommendation Using Knowledge Graph

In [1]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
from operator import itemgetter
import matplotlib.pyplot as plt
from IPython.display import display


In [2]:
resume_counts=3
jd_count=3

In [3]:
nx.__version__

'2.6.3'

In [4]:
job_description_dataset = pd.read_json(
    "Dataset\dataset_job_description.json")
resume_dataset = pd.read_json("Dataset\dataset_resume.json")


In [5]:
for resumeData in resume_dataset.values[:1] : # first two resumes
    print(resumeData[0]['id']) #id
    print(resumeData[0]['careerjunction_za_skills']) # skills
    print(resumeData[0]['careerjunction_za_courses']) # Degree 
    print(f" Experience {len(resumeData[0]['careerjunction_za_historical_jobtitles'])}") # Experience


1
['Emergency Handling', 'Microbiology', 'microsoft powerpoint', 'microsoft office', 'microsoft excel', 'microsoft project management', 'Microsoft word', 'Outlook', 'BootStrap', 'Internet explorer', 'Marketing/Sales', 'Quality Control', 'Quality Assurance', 'Research and development', 'Problem solving']
['Btech: Food Technology', 'National Diploma: Food Technology', 'Senior Certificate']
 Experience 5


In [6]:
for jobData in job_description_dataset[:1].values:  # first two job description
    print(jobData[17])  # Id
    print(jobData[0])  # Salary
    print(jobData[6])  # Skills
    print(jobData[2])  # Degree
    numbers = [int(word) for word in jobData[14].split() if word.isdigit()]
    numbers.append(0)
    experience = max(numbers) 
    print(experience) # Experience
    print(jobData[20])  # Job Type


2b387592-8148-5720-a661-a2730061d14c
$17.23 - $22.00 / hour
Microsoft Office , java ee , cics , Adobe Photoshop , asp , Adobe After Effects , Powerhouse Programming- beginners and advance , asp .NET
Bachelor's Degree
2
FULL_TIME


In [7]:
nodes = [n for n in resume_dataset.values][:resume_counts] # TODO: Change count to desired no of resumes

nodes_id=[n[0]['id'] for n in nodes]
# nodes_id

edges = [tuple(e) for e in job_description_dataset.values]
# edges

Extract Job Id, Skills and experience from job post

In [8]:
G = nx.Graph()

jobid = []
req_skills = []
req_experience = []

counter = 0

for job in job_description_dataset[:jd_count].values: # TODO: Change count to desired no of JD
    _skills = job[6].split(",")
    jobid.append(job[17])
    req_skills.append(_skills)
    numbers = [int(word) for word in job[14].split() if word.isdigit()]
    numbers.append(0)
    experience = max(numbers)
    req_experience.append(experience)
    counter += 1
    if(counter == jd_count):  # TODO: Change count to desired no of JD
        break


Extract resume Id, Skills and experience from Resume

In [9]:

resumeId = []
resumeSkills = []
resumeExperience = []
# Side note: use a for loop instead of individual loops

resumes = [n for n in resume_dataset.values][:resume_counts] # TODO: Change count to desired no of resumes
resumeId = [n[0]['id'] for n in resumes]
resumeSkills = [n[0]['careerjunction_za_skills'] for n in resumes]
resumeExperience = [
    len(n[0]['careerjunction_za_historical_jobtitles']) for n in resumes]
resumeExperience


[5, 5, 0]

In [10]:
edge_data = zip(jobid, req_skills, req_experience)
node_data = zip(resumeId, resumeSkills, resumeExperience)

Adding Job Description and skill node, are connected with an edge having weight of **experience**

In [11]:
for j, e in enumerate(edge_data):
    src = e[0]
    dst = e[1]
    w = e[2]
    # print(dst)
    G.add_node(src, color='#dd4b39', title=src,type='jobDescription')
    for eachSkill in dst:
        eachSkill=eachSkill.strip()
        G.add_node(eachSkill, title=dst, type='skill', experience=w)
        # if str(w).isdigit():
        if w is None:
            G.add_edge(src, eachSkill, value=w, color='#00ff1e', label=w)
        if 1 < w <= 70000:
            G.add_edge(src, eachSkill, value=w, color='#FFFF00', label=w)
        if w > 70000:
            G.add_edge(src, eachSkill, value=w, color='#dd4b39', label=w)

        else:
            G.add_edge(src, eachSkill) 


Adding resume and skill node, are connected with an edge

In [12]:
for j, e in enumerate(node_data):
    src = "ResumeId-" + str(e[0])
    dst = e[1]
    w = e[2]
    G.add_node(src, color='#dd4b39', type='resume', title=src)
    for eachSkill in dst:
        eachSkill = eachSkill.strip()
        G.add_node(eachSkill, title=eachSkill, type='skill', experience=w)
        G.add_edge(src, eachSkill, color='#00ff1e')

nt = Network('1080px', '1960px', notebook=False)
# populates the nodes and edges data structures
nt.from_nx(G)
nt.show('nx.html')


## Analysis of Network
#### Find Skills ,Job Description , Resume having highest degree. (which means highest connections)

In [13]:
degree_dict=dict(G.degree(G.nodes()))
nx.set_node_attributes(G,degree_dict,'degree')

sorted_degree= sorted(degree_dict.items(),key=itemgetter(1),reverse=True)

def popularNodeByType(type):
  for d in sorted_degree:
    if(G.nodes[d[0]]['type'] == type):
        print(f"{d}\n")
        return d;

print("Popular Skill:")
popular_skill=popularNodeByType('skill')

print("Popular Resume:")
popular_resume=popularNodeByType('resume')

print("Popular Description:")
popular_jd=popularNodeByType('jobDescription')



Popular Skill:
('Emergency Handling', 2)

Popular Resume:
('ResumeId-1', 15)

Popular Description:
('2b387592-8148-5720-a661-a2730061d14c', 8)



#### Eigen vector centrality take an account of nodes , edges as well as connections of that nodes

In [14]:
betweenness_dict = nx.betweenness_centrality(G,weight=True)  # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G,max_iter=600)  # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

sorted_betweenness = sorted(
    eigenvector_dict.items(), key=itemgetter(0), reverse=True)

print("Top 20 nodes by eigenvector centrality:")
count=0;
for b in sorted_betweenness:
    if(G.nodes[b[0]]['type']=='resume'):
        print(b)
        count+=1
    if count==20:
        break


Top 20 nodes by eigenvector centrality:
('ResumeId-3', 0.00010502478380464879)
('ResumeId-2', 0.23405726745902172)
('ResumeId-1', 0.6672459700286026)


**Input**: skill
**Output:** recommend (description,
        resumes)

In [15]:
allConnectedNodes=nx.node_connected_component(G, 'BootStrap')
for node in allConnectedNodes:
    print(node)

Quality Assurance
REST Services
microsoft powerpoint
ResumeId-1
SQL Databse Development
EntityFrameWork 6
Outlook
Quality Control
Problem solving
microsoft excel
Internet explorer
C#
SSRS
JQuery
MVC5
ResumeId-2
Marketing/Sales
BootStrap
Microsoft word
Microbiology
Emergency Handling
Research and development
SSIS
microsoft office
microsoft project management


#### Recommend Users to learn skills
[Bipertite graph concept applies here]

In [16]:
resume1_nbrs=G.neighbors('ResumeId-2')

resume3_nbrs=G.neighbors('ResumeId-3')

differentSkills= set(resume1_nbrs).difference(resume3_nbrs)
skillNodes=[]

for skill in differentSkills:
    skillNodes.append(G.nodes[skill])
    
sortedSkillsAccrToEigen = sorted(skillNodes, key=lambda i: i['eigenvector'],reverse=True)

print("Top Skills to be learned")
for skill in sortedSkillsAccrToEigen:
    print(f"{skill['title']} {round(skill['eigenvector'],3)}")


Top Skills to be learned
Emergency Handling 0.227
BootStrap 0.227
EntityFrameWork 6 0.059
C# 0.059
SSRS 0.059
JQuery 0.059
MVC5 0.059
REST Services 0.059
SSIS 0.059
SQL Databse Development 0.059


#### input: Resume 

#### output: List of recommended Job Descriptions

In [17]:
def get_all_adj_job_description_nodes(resume3_nbrs):
    related_job_description_nodes_dict = {}

    for skill in resume3_nbrs:
        skill_nabhour = G.neighbors(skill)
        for each_node in skill_nabhour:
            if(G.nodes[each_node]['type'] == 'jobDescription'):
                if G.nodes[each_node]['title'] in related_job_description_nodes_dict:
                    related_job_description_nodes_dict[G.nodes[each_node]
                                                       ['title']] += 1
                else:
                    related_job_description_nodes_dict[G.nodes[each_node]
                                                       ['title']] = 1

    return related_job_description_nodes_dict


resume3_nbrs = G.neighbors('ResumeId-20')
jobDescriptionNodes = get_all_adj_job_description_nodes(resume3_nbrs)

sortedJobDescriAccToConNodes = dict(sorted(
    jobDescriptionNodes.items(), key=lambda i: i[1], reverse=True))

print("Recomended Job Descriptions")
for job in sortedJobDescriAccToConNodes:
    print(f"{job} {sortedJobDescriAccToConNodes[job]}")


NetworkXError: The node ResumeId-20 is not in the graph.